In [19]:
import numpy as np
import matplotlib.pyplot as plt 
import keras as ke
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from keras.layers import *  
from sklearn.model_selection import train_test_split
from pylab import meshgrid
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from keras.initializers import he_normal, glorot_normal

In [36]:
def approximate_2d(y_func, epochs = 10, batch_size = 4, hidden_layers = [4], test_size = 0.5, init = 'glorot_normal', act = 'sigmoid'):
    x = np.arange(-10, 10, 0.1).reshape(-1, 1)
    y = y_func(x).reshape(-1, 1)
    
    unseen_x = np.arange(10, 20, 0.1).reshape(-1, 1)
    unseen_y = y_func(unseen_x).reshape(-1, 1)
    
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = test_size, shuffle = True)
    
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()
    
    train_x = scaler_x.fit_transform(train_x)
    test_x = scaler_x.transform(test_x)
    unseen_x = scaler_x.transform(unseen_x)
    
    train_y = scaler_y.fit_transform(train_y)
    test_y = scaler_y.transform(test_y)
    unseen_y = scaler_y.transform(unseen_y)
    
    if init == 'he_normal':
        init = he_normal(seed = 666)
    elif init == 'glorot_normal':
        init = glorot_normal(seed = 666)
    model = Sequential()  
    model.add(Input(shape=(1,)))
    for i, l in enumerate(hidden_layers):
        model.add(Dense(l, kernel_initializer = init, activation = act)) if i == 0 else\
        model.add(Dense(l, kernel_initializer = init, activation = act))
    
    model.add(Dense(1, kernel_initializer = init))
    model.compile(optimizer = 'sgd', loss = 'mse')
    model.fit(train_x, train_y, epochs = epochs, batch_size = batch_size, validation_split = 0.2, verbose=0)
    
    # Predictions
    preds = scaler_y.inverse_transform(model.predict(test_x)) # Preds on test data
    preds_train = scaler_y.inverse_transform(model.predict(train_x)) # Preds on train data
    unseen_preds = scaler_y.inverse_transform(model.predict(unseen_x)) # Preds on unseen data
    
    # Inverse transform of data
    unseen_x = scaler_x.inverse_transform(unseen_x)
    unseen_y = scaler_y.inverse_transform(unseen_y)
    train_x = scaler_x.inverse_transform(train_x)
    test_x = scaler_x.inverse_transform(test_x)
    
    # Plotting results
    fig = plt.figure(figsize = (19, 6))
    
    # Learning curves plot
    plt.subplot(121)
    H = model.history.history
    plt.plot(H['mse'], label = 'loss')
    plt.plot(H['val_loss'], label = 'val_loss')
    plt.grid(); plt.legend()
    
    # Predictions plot
    plt.subplot(122)
    plt.plot(x, y, label = '$f(x)$')
    plt.scatter(test_x, preds, label = 'Test_preds', s = 15, c = 'g', marker = 'x')
    plt.scatter(train_x, preds_train, label = 'Train_preds', s = 5, c = 'y', alpha = 0.9)
    plt.scatter(unseen_x, unseen_preds, label = 'unseen_preds', s = 10, c = 'k', marker = '1')
    plt.plot(unseen_x, unseen_y, label = 'Unseen data')
    plt.grid(); plt.legend()
    plt.title(f'{len(hidden_layers)} hidden: {hidden_layers} neurons, {act} activation')
    plt.show()  

In [ ]:
def sinusoid(x):
    return np.sin(x)

approximate_2d(sinusoid, hidden_layers = [16, 32, 64, 128], epochs = 500, act = 'relu', init = 'he_normal') 